## Web Scrapping no site Reclame Aqui
Web Automation com Selenium

Este projeto foi Baseado do projeto de Ana Paula Machado, onde foi possivel obter vários dados do reclame aqui.


## Importação dos Módulos

In [1]:
import pandas as pd
import numpy as np
import time
from datetime import datetime
from operator import concat


from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import requests

In [2]:
from datetime import datetime
dataextract = datetime.today().strftime('%d%m%Y')
databarra = datetime.today().strftime('%d/%m/%Y')
##print(datetime.today().strftime('%Y-%m-%d'))
#dataextract = print(datetime.today().strftime('%d%m%Y'))

## Coleta da base

In [3]:
# Importando os dados
consumidor = pd.read_csv('dados/baselista.csv', sep = ';')

In [4]:
# Visualizando as primeiras 5 linhas
consumidor.head()

,nome_empresa,link_reclame
0,SAMS CLUB - Loja Física,https://www.reclameaqui.com.br/empresa/sams-club/
1,BIG - Lojas Físicas,https://www.reclameaqui.com.br/empresa/big-hip...
2,Big Bompreço - Loja Física,https://www.reclameaqui.com.br/empresa/big-bom...
3,Nacional Supermercado,https://www.reclameaqui.com.br/empresa/naciona...
4,Maxxi Atacado,https://www.reclameaqui.com.br/empresa/maxxi-a...


In [5]:
# Obtendo o shape dos dados
consumidor.shape
consumidor.columns

# 143572 observações e 30 variáveis (colunas)

Index(['nome_empresa', 'link_reclame'], dtype='object')

In [6]:
consumidor.columns

Index(['nome_empresa', 'link_reclame'], dtype='object')

#### Web Scrapping Reclame Aqui

In [7]:
# Criando um novo df para armazenar apenas os valores únicos
df_scores = pd.DataFrame(consumidor['link_reclame'].unique(), columns = ['link_reclame'])
#df_scores = pd.DataFrame(consumidor['nome_empresa'].unique(), columns = ['Nome Empresa'])
df_scores

,link_reclame
0,https://www.reclameaqui.com.br/empresa/sams-club/
1,https://www.reclameaqui.com.br/empresa/big-hip...
2,https://www.reclameaqui.com.br/empresa/big-bom...
3,https://www.reclameaqui.com.br/empresa/naciona...
4,https://www.reclameaqui.com.br/empresa/maxxi-a...
5,https://www.reclameaqui.com.br/empresa/sams-lo...
6,https://www.reclameaqui.com.br/empresa/big-loj...
7,https://www.reclameaqui.com.br/empresa/big-bom...


In [8]:
# Criando uma nova coluna para alocar os scores e os nomes das empresas que aparecem no site do Reclame Aqui
df_scores['Nome no Reclame Aqui'] = ''
df_scores['Score Reclame Aqui'] = ''
df_scores['Reclamações respondidas'] = ''
df_scores['Voltariam a fazer negócio'] = ''
df_scores['Indice de solução'] = ''
df_scores['Nota do consumidor'] = ''
df_scores['Não respondidas'] = ''
df_scores['Quantidade Avaliadas'] = ''
df_scores['Quantidade respondidas'] = ''
df_scores['Quantidade de reclamacões'] = ''
df_scores['Data'] = ''
df_scores

,link_reclame,Nome no Reclame Aqui,Score Reclame Aqui,Reclamações respondidas,Voltariam a fazer negócio,Indice de solução,Nota do consumidor,Não respondidas,Quantidade Avaliadas,Quantidade respondidas,Quantidade de reclamacões,Data
0,https://www.reclameaqui.com.br/empresa/sams-club/,,,,,,,,,,,
1,https://www.reclameaqui.com.br/empresa/big-hip...,,,,,,,,,,,
2,https://www.reclameaqui.com.br/empresa/big-bom...,,,,,,,,,,,
3,https://www.reclameaqui.com.br/empresa/naciona...,,,,,,,,,,,
4,https://www.reclameaqui.com.br/empresa/maxxi-a...,,,,,,,,,,,
5,https://www.reclameaqui.com.br/empresa/sams-lo...,,,,,,,,,,,
6,https://www.reclameaqui.com.br/empresa/big-loj...,,,,,,,,,,,
7,https://www.reclameaqui.com.br/empresa/big-bom...,,,,,,,,,,,


In [9]:
# Abrindo o navegador
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

for i in range(len(df_scores['link_reclame'])):

    #Abrindo site
    driver.get(df_scores['link_reclame'][i])

    #Coletando dados
    elem0 = driver.find_element(By.XPATH, '//*[@id="hero"]/div[2]/div/div[2]/div[1]/h1')
    nome_reclame = BeautifulSoup(elem0.get_attribute("innerHTML"), "html.parser").get_text().strip()
    df_scores['Nome no Reclame Aqui'][i] = nome_reclame

    # Localizando o elemento com o score
    elem1 = driver.find_element(By.XPATH, '//*[@id="reputation"]/div[1]/div[1]/div[2]/span[2]/b')
    nome_score = BeautifulSoup(elem1.get_attribute("innerHTML"), "html.parser").get_text().strip()
    df_scores['Score Reclame Aqui'][i] = nome_score

    # Localizando reclamações respondidas
    elem2 = driver.find_element(By.XPATH, '//*[@id="reputation"]/div[2]/div[1]/div[1]/span')
    reclam_respond = BeautifulSoup(elem2.get_attribute("innerHTML"), "html.parser").get_text().strip()
    df_scores['Reclamações respondidas'][i] = reclam_respond

    # Localizando Voltariam a fazer negócio
    elem3 = driver.find_element(By.XPATH, '//*[@id="reputation"]/div[2]/div[1]/div[2]/span')
    volt_negoc = BeautifulSoup(elem3.get_attribute("innerHTML"), "html.parser").get_text().strip()
    df_scores['Voltariam a fazer negócio'][i] = volt_negoc

    # Localizando Indice de solução
    elem4 = driver.find_element(By.XPATH, '//*[@id="reputation"]/div[2]/div[1]/div[3]/span')
    indi_solu = BeautifulSoup(elem4.get_attribute("innerHTML"), "html.parser").get_text().strip()
    df_scores['Indice de solução'][i] = indi_solu
        
    # Localizando Nota do consumidor
    elem5 = driver.find_element(By.XPATH, '//*[@id="reputation"]/div[2]/div[1]/div[4]/span')
    nota_consu = BeautifulSoup(elem5.get_attribute("innerHTML"), "html.parser").get_text().strip()
    df_scores['Nota do consumidor'][i] = nota_consu

    # Localizando Não respondidas
    elem6 = driver.find_element(By.XPATH, '//*[@id="reputation"]/div[2]/div[2]/div[1]/a/div/b')
    nao_resp = BeautifulSoup(elem6.get_attribute("innerHTML"), "html.parser").get_text().strip()
    df_scores['Não respondidas'][i] = nao_resp

    # Localizando Quantidade Avaliadas
    elem7 = driver.find_element(By.XPATH, '//*[@id="reputation"]/div[2]/div[2]/div[2]/a/div/b')
    qtd_avaliad = BeautifulSoup(elem7.get_attribute("innerHTML"), "html.parser").get_text().strip()
    df_scores['Quantidade Avaliadas'][i] = qtd_avaliad

    # Localizando Quantidade respondidas
    elem8 = driver.find_element(By.XPATH, '//*[@id="reputation"]/div[1]/div[2]/a[2]/div/div/b')
    qtd_respondid = BeautifulSoup(elem8.get_attribute("innerHTML"), "html.parser").get_text().strip()
    df_scores['Quantidade respondidas'][i] = qtd_respondid

    # Localizando Quantidade de reclamacões
    elem9 = driver.find_element(By.XPATH, '//*[@id="reputation"]/div[1]/div[2]/a[1]/div/div/b')
    qtd_reclama = BeautifulSoup(elem9.get_attribute("innerHTML"), "html.parser").get_text().strip()
    df_scores['Quantidade de reclamacões'][i] = qtd_reclama

    df_scores['Data'][i] = databarra

driver.close() 



[WDM] - Downloading: 100%|██████████| 6.29M/6.29M [00:00<00:00, 11.9MB/s]


In [10]:
# Visualizando o dataframe CLASS_NAME
df_scores

,link_reclame,Nome no Reclame Aqui,Score Reclame Aqui,Reclamações respondidas,Voltariam a fazer negócio,Indice de solução,Nota do consumidor,Não respondidas,Quantidade Avaliadas,Quantidade respondidas,Quantidade de reclamacões,Data
0,https://www.reclameaqui.com.br/empresa/sams-club/,SAMS CLUB - Loja Física,6.9,98.3%,55.5%,70.7%,5.83,6,229,529,538,16/10/2022
1,https://www.reclameaqui.com.br/empresa/big-hip...,BIG - Lojas Físicas,7.2,99.6%,60.2%,68.7%,6.39,1,83,239,240,16/10/2022
2,https://www.reclameaqui.com.br/empresa/big-bom...,Big Bompreço - Loja Física,7.5,100%,71.1%,71.1%,6.32,0,38,111,111,16/10/2022
3,https://www.reclameaqui.com.br/empresa/naciona...,Nacional Supermercado,7.3,100%,71.4%,71.4%,5.93,0,14,38,38,16/10/2022
4,https://www.reclameaqui.com.br/empresa/maxxi-a...,Maxxi Atacado,6.6,100%,52.9%,64.7%,5.29,0,51,105,105,16/10/2022
5,https://www.reclameaqui.com.br/empresa/sams-lo...,SAMS - Loja Online,6.9,98.3%,57.6%,72.9%,5.35,9,295,705,717,16/10/2022
6,https://www.reclameaqui.com.br/empresa/big-loj...,BIG - Loja Online,7.0,98.4%,53.2%,77.6%,5.61,8,250,565,574,16/10/2022
7,https://www.reclameaqui.com.br/empresa/big-bom...,Big Bompreço - Loja Online,7.2,99.4%,56%,81%,5.51,0,84,156,157,16/10/2022


In [11]:
# Exportando a tabela
df_scores.to_excel('dados/dados_RA.xlsx')

In [12]:
##print(datetime.today().strftime('%Y-%m-%d'))

df_scores.to_excel(f'dados/Dados historicos/{dataextract}.xlsx')

In [13]:
df_scores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   link_reclame               8 non-null      object
 1   Nome no Reclame Aqui       8 non-null      object
 2   Score Reclame Aqui         8 non-null      object
 3   Reclamações respondidas    8 non-null      object
 4   Voltariam a fazer negócio  8 non-null      object
 5   Indice de solução          8 non-null      object
 6   Nota do consumidor         8 non-null      object
 7   Não respondidas            8 non-null      object
 8   Quantidade Avaliadas       8 non-null      object
 9   Quantidade respondidas     8 non-null      object
 10  Quantidade de reclamacões  8 non-null      object
 11  Data                       8 non-null      object
dtypes: object(12)
memory usage: 896.0+ bytes
